In [4]:
import pandas as pd
import re
import os
import fitz
from PIL import Image
import pytesseract

In [288]:
pdf_path = 'C1.pdf'

text = ''
pdf_document = fitz.open(pdf_path)
count = pdf_document.page_count
for i in range(count):
    page = pdf_document[i]
    tx= page.get_text()
    text+=tx

In [289]:
# print(text)
k1 = {}

In [290]:
patterns = [
    r'Taxable Sub Total\s+(\d+,\d+\.\d+)',
    # r'IGST\s+\d+% - \d+\%\s+(\d+,\d+\.\d+)',
    r'Non-Taxable Sub Total\s+(\d+,\d+\.\d+)',
    # r'Round Off\s+(\d+\.\d+)',
    r'TOTAL\s+₹\s+(\d+,\d+\.\d+)'
]

# Search for the patterns in the text
results = {}
for pattern in patterns:
    match = re.search(pattern, text)
    if match:
        results[match.group(0)] = match.group(1)

for i in list(results.keys()):
    a,b = i.split('\n')[0],i.split('\n')[1]
    k1[a] = b

In [291]:
k1

{'Taxable Sub Total': '4,258.00', 'TOTAL': '₹ 4,471.00'}

In [292]:
invoice_number_pattern = r'Invoice Number:\s*([A-Z0-9-]+)'
invoice_date_pattern = r'Invoice Date:\s*(\d{2}-\d{2}-\d{4})'

# Search for the patterns in the text
invoice_number_match = re.search(invoice_number_pattern, text)
invoice_date_match = re.search(invoice_date_pattern, text)

# Extract the information if found
invoice_number = invoice_number_match.group(1) if invoice_number_match else None
invoice_date = invoice_date_match.group(1) if invoice_date_match else None

# Print the extracted information
k1["Invoice Number"] =  invoice_number
k1["Invoice Date"] = invoice_date

In [293]:
pan_pattern = r'PAN:\s*([A-Z0-9]+)'
gstin_pattern = r'GSTIN:\s*([A-Z0-9]+)'


pan_match = re.search(pan_pattern, text)
gstin_match = re.search(gstin_pattern, text)


pan_number = pan_match.group(1) if pan_match else None
gstin_number = gstin_match.group(1) if gstin_match else None


k1["PAN"] =  pan_number
k1["GSTIN"] =  gstin_number

In [294]:
k1

{'Taxable Sub Total': '4,258.00',
 'TOTAL': '₹ 4,471.00',
 'Invoice Number': 'AF2223-026869',
 'Invoice Date': '15-03-2023',
 'PAN': 'AAFCI9896L',
 'GSTIN': '24AAFCI9896L1ZQ'}

In [295]:
# pdf_path = 'C1.pdf'

# text = ''
# pdf_document = fitz.open(pdf_path)
# count = pdf_document.page_count
# for i in range(count):
#     if i==0:
#         continue
#     page = pdf_document[i]
#     tx= page.get_text()
#     text+=tx

In [296]:
# print(text)

In [297]:
km_pattern = r'KM\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)'
time_pattern = r'Time\s+(\d{4} \d{2}/\d{2})\s+(\d{4} \d{2}/\d{2})\s+(\d{2}:\d{2})\s+(\d{2}:\d{2})'


km_match = re.search(km_pattern, text)
time_match = re.search(time_pattern, text)

# Extract information if matches are found
if km_match:
    km_start, km_end, km_total, km_extra = km_match.groups()
else:
    km_start, km_end, km_total, km_extra = None, None, None, None

if time_match:
    time_start, time_end, time_total, time_extra = time_match.groups()
else:
    time_start, time_end, time_total, time_extra = None, None, None, None

k =  {
    "KM Start": km_start,
    "KM End": km_end,
    "KM Total": km_total,
    "KM Extra": km_extra,
    "Time Start": time_start,
    "Time End": time_end,
    "Time Total": time_total,
    "Time Extra": time_extra
}

In [9]:
# k

In [4]:
import tabula

# Define the coordinates of the area containing the table [top, left, bottom, right]
area = [110,33, 550, 550]  


def extract_data_from_pdf(pdf_path, area):
    try:
        # Read the PDF using Tabula with the defined area
        tables = tabula.read_pdf(pdf_path, pages='all', area=area)
        return tables
    except Exception as e:
        print(f"Error: {e}")
        return None


pdf_path = "C1.pdf"

# Extract data from PDF using defined area
tables = extract_data_from_pdf(pdf_path, area)
df1 = tables[1]
df1.fillna('', inplace=True)
# df1

In [301]:
ls = list(df1.columns)
k = {}
for i in ls:
    k[df1[i].iloc[6].replace('\r',' ')]= df1[i].iloc[7].replace('\r',' ')

In [6]:
import tabula

# Define the coordinates of the area containing the table [top, left, bottom, right]
area = [111,20, 550, 550]  

def extract_data_from_pdf(pdf_path, area):
    try:
        # Read the PDF using Tabula with the defined area
        tables = tabula.read_pdf(pdf_path, pages='all', area=area)
        return tables
    except Exception as e:
        print(f"Error: {e}")
        return None

# PDF file path
pdf_path = "C1.pdf"


tables = extract_data_from_pdf(pdf_path, area)
df1 = tables[1]
df1.fillna('', inplace=True)

# ls = list(df1.columns)
# for i in ls[0:2]:
#     k[df1[i].iloc[7].replace('\r',' ')]= df1[i].iloc[8].replace('\r',' ')

In [7]:
df1

,Passengers Dipendra Singh,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,Reporting Address Chandigarh Hotel (Please con...,,,,,,,,,,,
1,Drop Address Field work at Chandigarh and drop...,,,,,,,,,,,
2,Scanned Dutyslip View Duty Slip,,,,,,,,,,,
3,Start,End,,,,,Total,,,Extra,,
4,KM 157037,157144,,,,,107,,,27,,
5,Time 0915 11/11,2030 11/11,,,,,11:15,,,03:15,,
6,Additional Charges:,,,,,,,,,,,
7,Charges,Amount,,,,Receipt,,,,,,
8,TOLL,100.00,,,,View Receipt,,,,,,
9,Customer signature not available.,,,,,,,,,,,


In [226]:
import tabula

# Define the coordinates of the area containing the table [top, left, bottom, right]
area = [0,10, 400, 300]  


def extract_data_from_pdf(pdf_path, area):
    try:
        # Read the PDF using Tabula with the defined area
        tables = tabula.read_pdf(pdf_path, pages='all', area=area)
        return tables
    except Exception as e:
        print(f"Error: {e}")
        return None

# PDF file path
pdf_path = "C1.pdf"

# Extract data from PDF using defined area
tables = extract_data_from_pdf(pdf_path, area)
df1 = tables[1]
df1.fillna('', inplace=True)



In [233]:
Date = df1.loc[0]['Duty #10722998-1'].split(' ')[-1]
driver = df1.loc[3]['Duty #10722998-1'].split(' ')[-1]
reporting_address = (' ').join([s for s in df1.loc[5]['Duty #10722998-1'].split(' ')[2:]])
drop_address = (' ').join([s for s in df1.loc[6]['Duty #10722998-1'].split(' ')[2:]])

In [236]:
k['Date'] = Date
k['driver'] = driver
k['reporting_address'] = reporting_address
k['drop_address'] = drop_address

In [283]:
k.update(k1)

In [285]:
# k

In [287]:
df1 = pd.DataFrame([k])
df1.to_excel('cus_1.xlsx')
df1

,Duty Id,Booked By,Passenger,Veh.Group,Veh.Num.,Duty Type,Qty,S.Date,E.Date,S.Time,...,Date,driver,reporting_address,drop_address,Taxable Sub Total,TOTAL,Invoice Number,Invoice Date,PAN,GSTIN
0,#10722998- 1,Harmeet Singh,Dipendra Singh,CRYSTA,PB01N0768,LOCAL 8HRS/80KM,1,11-11,11-11,0915,...,11-11-2022,Satpal,Chandigarh Hotel (Please confirm with guest),Field work at Chandigarh and drop at hotel,"5,296.00","₹ 5,661.00",AF2223-026868,15-03-2023,AAFCI9896L,24AAFCI9896L1ZQ
